In [51]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import csv
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import MinMaxScaler
from ta import add_all_ta_features
import torch.nn.functional as F

In [53]:
# Гиперпараметры
input_size = 95
target_col_name = "close"
N = 30

In [55]:
import torch
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
import joblib
from torch.utils.data import Dataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Словарь для хранения обработанных датафреймов
all_datasets = {}
all_scalers = {}

# Проверка наличия сохраненных данных и скейлеров
processed_data_exists = os.path.exists('all_datasets.pkl')
scalers_exist = os.path.exists('all_scalers.pkl')

if processed_data_exists and scalers_exist:
    all_datasets = joblib.load('all_datasets.pkl')
    all_scalers = joblib.load('all_scalers.pkl')
    print("Загружены ранее сохраненные обработанные данные и скейлеры.")
else:
    datasets_path = "datasets"
    for file_name in os.listdir(datasets_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(datasets_path, file_name)
            dataset_name = os.path.splitext(file_name)[0]
            
                        # Пробуем разные кодировки для чтения файла
            encodings = ['utf-8', 'latin1', 'cp1251', 'iso-8859-1']
            
            for encoding in encodings:
                try:
                    df = pd.read_csv(file_path, header=None, skiprows=1, encoding=encoding)
                    break
                except UnicodeDecodeError:
                    continue
            df.drop(columns=[1, 2], inplace=True)
            df.columns = ['unix', 'open', 'high', 'low', 'close', 'VolumeBTC', 'VolumeUSD']
            
            if df.isnull().values.any():
                print(f"Обнаружены пропущенные значения в {file_name}.")
                rows_with_missing_values = df[df.isnull().any(axis=1)]
                print("Номера строк с пропущенными значениями:")
                print(rows_with_missing_values.index)
            else:
                print(f"Пропущенных значений в {file_name} не обнаружено.")
            
            df.dropna(inplace=True)
'''
Блок аугментации данных
'''
            # Масштабирование признаков
            scalers = {}
            for column in df.columns:
                scaler = MinMaxScaler()
                df[column] = scaler.fit_transform(df[column].values.reshape(-1, 1))
                scalers[column] = scaler
            
            all_datasets[dataset_name] = df
            all_scalers[dataset_name] = scalers
            
    
    joblib.dump(all_scalers, 'all_scalers.pkl')
    joblib.dump(all_datasets, 'all_datasets.pkl')
    print("Данные успешно обработаны и сохранены.")

# Получаем первый датасет из словаря
dataset_name = list(all_datasets.keys())[0]
df = all_datasets[dataset_name]

print(f"Используется датасет: {dataset_name}")

# Вычисляем точки разделения на основе длины датафрейма
total_rows = len(df)
train_size = int(0.8 * total_rows)
val_size = int(0.1 * total_rows)

# Разделяем датафрейм с сохранением временного порядка
train_df = df[:train_size]                           # Первые 80% данных
test_df = df[train_size:train_size + val_size]      # Следующие 10% данных
val_df = df[train_size + val_size:]                 # Последние 10% данных


Пропущенных значений в Bitstamp_test.csv не обнаружено.
Данные успешно обработаны и сохранены.


C:\ProgramData\anaconda3\Lib\site-packages\ta\trend.py:988: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '96443.42' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self._psar.iloc[i] = self._psar.iloc[i - 1] + (


In [7]:
class LSTM_CNN_Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, N, kernel_size, num_filters, dropout, seq_length):
        super(LSTM_CNN_Model, self).__init__()
        
        # LSTM layers
        self.lstm1 = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.init_weights(self.lstm1)
        
        # CNN layers
        self.conv1d = nn.Conv1d(in_channels=hidden_size, out_channels=num_filters, kernel_size=kernel_size)
        nn.init.xavier_uniform_(self.conv1d.weight)
        
        self.prelu = nn.PReLU()  # Изменение здесь
        
        # Second LSTM layer
        self.lstm2 = nn.LSTM(num_filters, hidden_size, num_layers, batch_first=True)
        self.init_weights(self.lstm2)
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, N)
        nn.init.xavier_uniform_(self.fc.weight)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
    
    def init_weights(self, layer):
        for name, param in layer.named_parameters():
            if 'weight_ih' in name:
                nn.init.kaiming_uniform_(param.data, nonlinearity='relu')
            elif 'weight_hh' in name:
                nn.init.orthogonal_(param.data)
            elif 'bias' in name:
                param.data.fill_(0)
    
    def forward(self, x):
        # First LSTM forward pass
        lstm_out, _ = self.lstm1(x)
        
        # Transpose the output to fit the Conv1d input shape
        lstm_out = lstm_out.permute(0, 2, 1)
        
        # CNN forward pass
        cnn_out = self.conv1d(lstm_out)
        cnn_out = self.prelu(cnn_out)  # Изменение здесь
        
        # Transpose the output to fit the second LSTM input shape
        cnn_out = cnn_out.permute(0, 2, 1)
        
        # Second LSTM forward pass
        lstm_out, _ = self.lstm2(cnn_out)
        
        # Dropout
        lstm_out = self.dropout(lstm_out)
        
        # Fully connected layer
        lstm_out_last = lstm_out[:, -1, :]
        output = self.fc(lstm_out_last)
        output = self.prelu(output)  # Изменение здесь
        
        return output.view(output.shape[0], N)  # Reshape the output to (batch_size, N)


МОДЕЛЬ ОБУЧАЕТСЯ ДАЛЕЕ

In [9]:
# Загрузка сохраненных данных и скейлеров
all_datasets = joblib.load('all_datasets.pkl')
all_scalers = joblib.load('all_scalers.pkl')

# Выбор конкретного датасета 
dataset_name = list(all_datasets.keys())[0]
df = all_datasets[dataset_name]
scalers = all_scalers[dataset_name]

# Получаем нужный скейлер для целевой переменной
target_scaler = scalers['close'] 
target_col_name = 'close'


In [ ]:
import joblib
import json
import torch
import numpy as np
import matplotlib.pyplot as plt
import os
from torch import nn

# Выбор конкретного датасета
dataset_name = list(all_datasets.keys())[0]
df = all_datasets[dataset_name]
scalers = all_scalers[dataset_name]

# Получение скейлера для целевой переменной
target_scaler = scalers['close']
target_col_name = 'close'

def data_generator(df, target_col_name, seq_length, batch_size, N):
    """
    Генератор для создания пакетов данных для LSTM_CNN модели
    """
    target_col_index = df.columns.get_loc(target_col_name)
    data = df.values
    data = data[:seq_length]
    
    while True:
        batch_data = []
        batch_labels = []
        
        for i in range(batch_size):
            if seq_length <= data.shape[0]:
                sequence = data[0:seq_length]
                target = data[-1, target_col_index]
            else:
                return
            batch_data.append(sequence)
            batch_labels.append(target)
        
        yield np.array(batch_data), np.array(batch_labels)
print('Начало работы нейросети. Это может занять некоторое время.')
# Загрузка лучших параметров и модели
with open('best_params.json', 'r') as f:
    best_params = json.load(f)

input_size = 95
best_model = LSTM_CNN_Model(input_size, best_params['hidden_size'], 
                           best_params['num_layers'], N, 
                           best_params['kernel_size'], 
                           best_params['num_filters'], 
                           best_params['dropout'], 
                           best_params['seq_length']).to(device)
best_model.load_state_dict(torch.load('best_model.pth'))
best_model.eval()

# Генерация предсказаний
val_gen = data_generator(df, target_col_name, seq_length=best_params['seq_length'], batch_size=1, N=N)

with torch.no_grad():
    for i, (inputs, labels) in enumerate(val_gen):
        if i >= 1:
            break
        inputs = torch.from_numpy(inputs).float().to(device)
        outputs = best_model(inputs)
        predicted_values = outputs.detach().cpu().numpy()
        predicted_values_scaled_back = target_scaler.inverse_transform(predicted_values)

# Получение значений из генератора
val_gen = data_generator(df, target_col_name, seq_length=best_params['seq_length'], batch_size=1, N=N)
for i, (inputs, labels) in enumerate(val_gen):
    if i >= 1:
        break
    generator_values = inputs[0, :, 0]

generator_values_scaled_back = target_scaler.inverse_transform(generator_values.reshape(-1, 1))

# Создание финального графика
plt.figure(figsize=(12, 6))
plt.plot(generator_values_scaled_back, label='Исходные значения', color='b')
plt.plot(range(len(generator_values_scaled_back)-1, len(generator_values_scaled_back)-1+len(predicted_values_scaled_back[0])), 
         predicted_values_scaled_back[0], 
         label='Предсказанные значения', color='r')
plt.legend()
plt.title('Сравнение исходных и предсказанных значений')
plt.xlabel('Временной шаг')
plt.ylabel('Значение')

# Создание директории outputs, если она не существует
os.makedirs('outputs', exist_ok=True)

# Сохранение графика
plt.savefig('outputs/график_предсказаний.png')
plt.close()
print('График предсказаний успешно сохранен в папку outputs.')